In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def rot_aut(Xtr,ytr):
    # aug on train
    data_cnt = len(ytr)
    print(data_cnt)
    aug_X = []
    aug_y = []

    for i in range(data_cnt):
        img = Xtr[i]
        tmp_y = ytr[i]

        # org img
        aug_X.append(img)
        aug_y.append(tmp_y)

        # flip
        tmp_img = np.fliplr(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.flipud(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.rot90(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)
    return np.array(aug_X),np.array(aug_y)

In [5]:
from keras.callbacks import ModelCheckpoint
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')

model model


In [6]:
from sklearn.model_selection import KFold
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        curr_x,curr_y = rot_aut(curr_x,curr_y)
        val_x,val_y = train_x[test_index],y[test_index]
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        model.fit(curr_x,curr_y,
                  validation_data=(val_x,val_y),
                  batch_size=16, epochs=30, 
                  verbose=2,
                  callbacks=[model_chk]
                 )
        model = load_model(model_p)
        
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

1203
Train on 4812 samples, validate on 401 samples
Epoch 1/30
Epoch 00001: val_loss improved from inf to 0.55307, saving model to best_m.h5
 - 6s - loss: 0.6291 - acc: 0.6394 - val_loss: 0.5531 - val_acc: 0.7207
Epoch 2/30
Epoch 00002: val_loss improved from 0.55307 to 0.38497, saving model to best_m.h5
 - 5s - loss: 0.4855 - acc: 0.7641 - val_loss: 0.3850 - val_acc: 0.8204
Epoch 3/30
Epoch 00003: val_loss improved from 0.38497 to 0.35179, saving model to best_m.h5
 - 5s - loss: 0.4111 - acc: 0.8013 - val_loss: 0.3518 - val_acc: 0.8504
Epoch 4/30
Epoch 00004: val_loss did not improve
 - 5s - loss: 0.3659 - acc: 0.8288 - val_loss: 0.3648 - val_acc: 0.8429
Epoch 5/30
Epoch 00005: val_loss did not improve
 - 5s - loss: 0.3525 - acc: 0.8373 - val_loss: 0.4125 - val_acc: 0.8180
Epoch 6/30
Epoch 00006: val_loss improved from 0.35179 to 0.26579, saving model to best_m.h5
 - 5s - loss: 0.3306 - acc: 0.8456 - val_loss: 0.2658 - val_acc: 0.8753
Epoch 7/30
Epoch 00007: val_loss did not improve
 

Epoch 2/30
Epoch 00002: val_loss improved from 0.54629 to 0.46267, saving model to best_m.h5
 - 6s - loss: 0.5256 - acc: 0.7276 - val_loss: 0.4627 - val_acc: 0.7681
Epoch 3/30
Epoch 00003: val_loss improved from 0.46267 to 0.36135, saving model to best_m.h5
 - 6s - loss: 0.4506 - acc: 0.7710 - val_loss: 0.3613 - val_acc: 0.8504
Epoch 4/30
Epoch 00004: val_loss did not improve
 - 6s - loss: 0.4022 - acc: 0.8069 - val_loss: 0.3872 - val_acc: 0.8204
Epoch 5/30
Epoch 00005: val_loss improved from 0.36135 to 0.29350, saving model to best_m.h5
 - 6s - loss: 0.3629 - acc: 0.8275 - val_loss: 0.2935 - val_acc: 0.8728
Epoch 6/30
Epoch 00006: val_loss did not improve
 - 6s - loss: 0.3476 - acc: 0.8383 - val_loss: 0.3114 - val_acc: 0.8703
Epoch 7/30
Epoch 00007: val_loss improved from 0.29350 to 0.28031, saving model to best_m.h5
 - 6s - loss: 0.3338 - acc: 0.8477 - val_loss: 0.2803 - val_acc: 0.8753
Epoch 8/30
Epoch 00008: val_loss did not improve
 - 5s - loss: 0.3066 - acc: 0.8572 - val_loss: 0.

In [7]:
import pickle
with open('../features/cnn_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_sub.csv', index=False)
# batch 64 0.274
# batch 16 0.268
# batch 32

0.26868347006
         id  is_iceberg
0  5941774d    0.448910
1  4023181e    0.623780
2  b20200e4    0.272541
3  e7f018bb    0.977164
4  4371c8c3    0.716347
